# Managing Ansys-CFX cases

In [ ]:
from cfdtoolkit.cfx.case import CFXCase

from cfdtoolkit import set_loglevel
set_loglevel('INFO')

Check if Ansys can be found:

In [ ]:
from cfdtoolkit.cfx import CFXInstallation

cfx = CFXInstallation('cfx.env')
cfx.pre

## Run and manage a steady-state CFX Case

The important exe-files are represented by python classes:
- `CFXSolve` for cfx5solve
- `CFXPre` for cfx5pre

Other interface classes are `CCLFile` to interface with .ccl-files and .h5ccl-files (The .ccl file as hdf5 file) and `CFXCase` to interface with all relevant files of a case.

In [ ]:
from cfdtoolkit.cfx.solve import CFXSolve
from cfdtoolkit.cfx.pre import CFXPre
from cfdtoolkit.cfx.ccl import CCLFile
from cfdtoolkit.cfx.case import CFXCase

Let's copy a .cfx file from the testdata directy here:

In [ ]:
import shutil
cfx_filename = shutil.copy2('../../testdata/cylinderflow/steady_state/cyl_steadystate_laminar.cfx',
                            'cyl_steadystate_laminar.cfx')

## Interface with a case:

The `CFXCase` class let's you keep track of all files associated to the case

In [ ]:
case = CFXCase(cfx_filename)
case.info()

Deleting old res files (should not exist, but let's call it anyhow):

In [ ]:
case.reset()

## Start from default values:

First we need to write the definition file:

In [ ]:
case.pre.write_def()
case.solve.run(nproc=4)
case.info()

Let's have a look into the .out-file:

In [ ]:
case.res_files[-1].outfile.data.cpu_seconds.plot()

## Change (some of the) settings
This is an optional setp. We could also directyl resume the simulation (see next section). However, sometimes we want to adjust some settings like timestep, for instance. Not everything can be done through this package, but most (TODO: Let the user pass/call a session file/seesion code)

In [ ]:
ccl = case.solve.write_ccl()
ccl

In [ ]:
ccl.flow[0]['SOLVER CONTROL/CONVERGENCE CONTROL'].options['Maximum Number of Iterations'] = '2'
ccl.flow[0]['SOLVER CONTROL/CONVERGENCE CONTROL'].options['Maximum Number of Iterations']

Change an expression:

In [ ]:
ccl.expressions.options

In [ ]:
ccl.expressions.options['Um'] = '2.3 [m/s]'
ccl.expressions.options['Um']

In [ ]:
case.pre.from_h5ccl(ccl.filename)
case.pre.write_def()

## Continue/Restart the case

In [ ]:
case.solve.run(nproc=4, ini_filename=case.res[-1])

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
for r in case.res:
    r.outfile.data.cpu_seconds.plot()
plt.show()

## Miscellaneous

### Mesh information:

In [ ]:
case.res_files[-1].outfile.get_mesh_info()

In [ ]:
------------ old:

In [ ]:
import pathlib
inlet_data_source = pathlib.Path("../../testdata/cylinderflow/inlet_profile_Re20.csv").resolve()
case_steady.ccl['LIBRARY/CEL/FUNCTION: inlet/DATA SOURCE'].options['File Name'] = str(inlet_data_source)
case_steady.ccl.dump()
case_steady.ccl.filename

In [ ]:
case_steady.ccl.flow[0].dump()

In [ ]:
del case_steady.ccl.expressions.options['dmfr']
del case_steady.ccl.flow[0].monitor_object['dmfr']

In [ ]:
if True:
    case_steady.start(initial_result_file=None, nproc=2, timeout=120, wait=True)

### Resume a case
You can call `.resume()` on your latest result to continue from it. Note, calling `resume` on the latest result file will not take changes in the ccl file into account! Calling it from the `CFXCase` instance however will! You may call `.update_from_ccl()` to be absolutely sure!

In [ ]:
case_steady.update_from_ccl()
case_steady.latest.resume(nproc=2)

# # or call:
# case_steady.resume(nproc=2)

### Stopping a case
You can call `.stop()` from your `CFXCase` instance to stop the run(s). This will btw. stop all running cases in the working directory!

In [ ]:
case_steady.stop()  # touches a file "stp" in the *.dir directory if exists

## Monitor Points and Expressions

User points are very helpful to analyze the runs

### Define an expression:

In [ ]:
case_steady.ccl.expressions.options['test'] = 'test'

In [ ]:
del case_steady.ccl.expressions.options['test']
del case_steady.ccl.expressions.options['dmfr']  # delete if exists from previous run for the sake of this tutorial

In [ ]:
case_steady.ccl.expressions.options['dmfr'] = 'massFlow()@OUTLET-massFlow()@INLET'

In [ ]:
case_steady.ccl.expressions.options

### Defining a monitor point:

In [ ]:
from cfdtoolkit.cfx.core import MonitorObject
case_steady.ccl.flow[0].monitor_object['dmfr'] = MonitorObject(expression_value='dmfr', coord_frame='Coord 0')
case_steady.ccl.dump()

Let's continue/resume the run as we have now a new expression set and registered it as a monitor point

In [ ]:
case_steady.resume(nproc=2)

### User Points

The example case has some user points registered which can be requested by calling the `Monitor` instance of e.g. the lates result file:

In [ ]:
case_steady.latest

In [ ]:
case_steady.latest.monitor.user_points.keys()

In [ ]:
case_steady.latest.monitor.user_points['dmfr'].plot()

In [ ]:
#if case_steady.latest:
print(len(case_steady.result_files.cfx_res_files))
print(case_steady.latest)

## Controll and change case settings

The following shows how to set and read various options in your case file.

In [ ]:
case_steady.ccl

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

In [ ]:
case_steady.ccl.to_ccl('test.ccl')

In [ ]:
#case_steady.import_ccl()

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

### Setting a new boundary condition

In [ ]:
from cfdtoolkit.cfx.boundary_conditions.inlet import NormalSpeed

In [ ]:
inlet = case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet')

In [ ]:
inlet.condition = NormalSpeed(flow_regime='Subsonic', normal_speed=1)

In [ ]:
inlet.condition

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

In [ ]:
case_steady.update()